
   VERSIÓN 1.0: FICHERO QUE SE ENCARGA DE GENERAR LOS TFRECORDS                                                                                                                      


Añadimos todos los imports necesarios para que nuestros ficheros funcionen

In [1]:
import numpy as np 
import scipy.io as sio
import os, os.path
import sys
import tensorflow as tf
import re
import glob
import SimpleITK as sitk
import scipy.ndimage
from PIL import Image
from random import shuffle
import nibabel as nib

Ruta de los ficheros que contienen nuestras imagenes médicas de los pacientes, para los ficheros "tfrecords" de entrenamiento y validación

In [2]:
path_tfrecords_train = os.path.join("C:", "\\Users\\rober\\Practicas\\training\\training_v2.tfrecords")
print(path_tfrecords_train)
path_tfrecords_validation = os.path.join("C:", "\\Users\\rober\\Practicas\\training\\validation_v2.tfrecords")
print(path_tfrecords_validation)

C:\Users\rober\Practicas\training\training_v2.tfrecords
C:\Users\rober\Practicas\training\validation_v2.tfrecords


Función encargada de mostrar el progreso de la lectura de imagenes de cada canal(t1,t2,t1ce,flair) de cada paciente del número total de pacientes que tengamos para cada conjunto de datos(training,validation)

In [3]:
def print_progress(count, total):
    # Percentage completion.
    pct_complete = float(count) / total

    # Status-message.
    # Note the \r which means the line should overwrite itself.
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

Parte de código encargada de obtener los path relativos al HGG y al LGG, y a continuación realizar el congunto de datos para cada parte

Hábria que hacer que esto fuera totalmente aleatorio, añadir el conjunto de datos para el test y aplicar porcentajes

In [4]:
directorioHGG = glob.glob('C:\\Users\\rober\\Practicas\\MICCAI_BraTS17_Data_Training\\HGG\\*')
directorioLGG = glob.glob('C:\\Users\\rober\\Practicas\\MICCAI_BraTS17_Data_Training\\LGG\\*')
print("Número de pacientes en el directorio HGG:",len(directorioHGG))
print("Número de pacientes en el directorio LGG:",len(directorioLGG))
print("\n")
directorio_entrenamiento = directorioHGG[:-209] + directorioLGG[:-74]
directorio_validation = directorioHGG[-1:]+directorioLGG[-1:]
print("DIRECTORIO DE ENTRENAMIENTO")
print("Índice,Directorio Paciente")
for i in enumerate(directorio_entrenamiento):
    print(i)
print("\n")
print("DIRECTORIO DE VALIDATION")
print("Índice,Directorio Paciente")
for i in enumerate(directorio_validation):
    print(i)

Número de pacientes en el directorio HGG: 210
Número de pacientes en el directorio LGG: 75


DIRECTORIO DE ENTRENAMIENTO
Índice,Directorio Paciente
(0, 'C:\\Users\\rober\\Practicas\\MICCAI_BraTS17_Data_Training\\HGG\\Brats17_2013_10_1')
(1, 'C:\\Users\\rober\\Practicas\\MICCAI_BraTS17_Data_Training\\LGG\\Brats17_2013_0_1')


DIRECTORIO DE VALIDATION
Índice,Directorio Paciente
(0, 'C:\\Users\\rober\\Practicas\\MICCAI_BraTS17_Data_Training\\HGG\\Brats17_TCIA_608_1')
(1, 'C:\\Users\\rober\\Practicas\\MICCAI_BraTS17_Data_Training\\LGG\\Brats17_TCIA_654_1')


Conjunto de funciones definidas para el autor necesarias para realizar la conversión de los datos a "TFRecords"

In [5]:
def norm_image_by_patient(imname):
        im = sitk.GetArrayFromImage(sitk.ReadImage(imname)).astype(np.float32)
        return (im - im.mean()) / im.std()
        roi_index = im > 0
        mean = im[roi_index].mean()
        std = im[roi_index].std()
        im[roi_index] -= mean
        im[roi_index] /= std
        return im

In [6]:
def checkLabel(label, d):
        if np.count_nonzero(label[d]) > 0:
            return True, 1
        else:
            return False, 0

In [7]:
def _processing_image(seq, label, depth):
        seqs = []
        labs = []
        for d in [depth-2, depth-1, depth]:
            label_data = label[d]
            labs.append(np.array(label_data))
            mod = []
            for im in seq:
                image_data = im[d]
                #image_data = scipy.ndimage.interpolation.zoom(image_data, 2, order=1, mode='nearest')
                # upsample 
                mod.append(image_data)
            seqs.append(np.array(mod))
        seqs = np.array(seqs)
        labs = np.array(labs)
        return seqs.tobytes(), labs.tobytes()

In [8]:
def _processing_image_single(seq, label, depth):
        mod = []
        for im in seq:
            image_data = im[depth]
            #image_data = scipy.ndimage.interpolation.zoom(image_data, 2, order=1, mode='nearest')
            # upsample 
            mod.append(image_data)
        mod = np.array(mod)
        label_data = label[depth]
        # upsample
        #label_data = scipy.ndimage.interpolation.zoom(label_data, 2, order=1, mode='nearest')
        shape = list(mod.shape)
        #if (mod.shape[0] != 4 or mod.shape[1] != 240 or mod.shape[2] != 240):
        #	print(shape)
        #	print("SHIT")
        #	exit()
        # 4,240,240
        return mod.tobytes(), label_data.tobytes(), shape

In [9]:
## SythText datasets is too big to store in a record. 
## So Transform tfrecord according to dir name

def _convert_to_example(image_data, label):
        #print 'shape: {}, height:{}, width:{}'.format(shape,shape[0],shape[1])
        example = tf.train.Example(features=tf.train.Features(feature={
                'image/encoded': bytes_feature(image_data),
                'label/encoded': bytes_feature(label)
                }))
        return example

Función auxiliar para encapsular un float y guardarlo en el archivo TFRecords.

In [10]:
def float_feature(value):
    """Wrapper for inserting float features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    if (type(value[0]) == np.float32):
        value = [v.item() for v in value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

Función auxiliar para encapsular los bytes y guardarlos en el archivo TFRecords.

In [11]:
def bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

Función encargada de generar los "TFRecords para el conjunto del Training"

In [12]:
# Abre un TFRecordWriter para el archivo de salida
with tf.python_io.TFRecordWriter(path_tfrecords_train) as writer:
    #Lista donde almacenaremos todos los objetos "Example"
    all_example = []
    #Bucle que se encarga de recorrer los pacioentes
    for i,path in enumerate(directorio_entrenamiento):
        #Mostramos por pantalla el progreso que llevamos 
        print_progress(count = i, total = len(directorio_entrenamiento)-1)
        print("\n")
        #Mostramos el Indice y el paciente actual
        print(i,path)
        #Como índica su nombre es el directorio del paciente actual
        directorio_paciente = glob.glob(path + '\\*')
        #Bucle que se encarga de recorrer los ficheros del paciente actual
        for(j,path_paciente) in enumerate(directorio_paciente):
            #print(j,path_paciente)
            #Leemos el fichero relacionado con el tipo "Flair"
            if path_paciente.endswith("flair.nii.gz"):
                #print("Entra en flair")
                flair = glob.glob(path_paciente)
                norm_flair = norm_image_by_patient(flair[0])
            #Leemos el fichero relacionado con el tipo "T2"
            if path_paciente.endswith("t2.nii.gz"):
                #print("Entra en t2")
                t2 = glob.glob(path_paciente)
                norm_t2 = norm_image_by_patient(t2[0])
            #Leemos el fichero relacionado con el tipo "T1"
            if path_paciente.endswith("t1.nii.gz"):
                #print("Entra en t1")
                t1 = glob.glob(path_paciente)
                norm_t1 = norm_image_by_patient(t1[0])
            #Leemos el fichero relacionado con el tipo "SEG"
            #Este es especial porque es la etiqueta donde están la segmentación hecha por el médico
            if path_paciente.endswith("seg.nii.gz"):
                #print("Entra en seg")
                label = glob.glob(path_paciente)[0]
                label = sitk.GetArrayFromImage(sitk.ReadImage(label)).astype(np.float32)
                #Leemos el fichero relacionado con el tipo "T1CE"
            if path_paciente.endswith("t1ce.nii.gz"):
                #print("Entra en t1ce")
                t1ce = glob.glob(path_paciente)
                norm_t1ce = norm_image_by_patient(t1ce[0])
        #Comprobamos que cada canal se haya normalizado correctamente 
        if((len(norm_flair) == 0) or (len(norm_t2) == 0) or (len(norm_t1) == 0) or (len(norm_t1ce) == 0)):
            seq = [norm_flair,norm_t2,norm_t1,norm_t1ce]
        else:
            ind = 0
            for depth in range(2,155):
                #Comprobamos las etiquetas
                is_valid, sample_num = checkLabel(label, depth)
                if ( not is_valid):
                    continue
                for i in range(sample_num):
                    #En este bucle procesamos las imagenes
                    image_data, label_data = _processing_image(seq, label, depth)
                    #Convertimos las imagenes y las etiquetas procesadas en el objetos de tipo "Example"
                    example = _convert_to_example(image_data, label_data)
                    #Indexamos el objeto convertido a la lista de objetos
                    all_example.append(example)
                #print("All_example_len",len(all_example))
        #Mostramos los cortes
        print("Cortes:", len(all_example))
        #Barajamos los cortes de forma aleatoria
        shuffle(all_example)
        #Bucle encargado de serializar la información y escribrla en un fichero "TFRecord"
        for ex in all_example:
            writer.write(ex.SerializeToString()) 
    print ('Transformación Realizada Completamente!!!')

- Progress: 0.0%

0 C:\Users\rober\Practicas\MICCAI_BraTS17_Data_Training\HGG\Brats17_2013_10_1


NameError: name 'seq' is not defined

Función encargada de generar los "TFRecords para el conjunto del Validation"

In [ ]:
# Abre un TFRecordWriter para el archivo de salida
with tf.python_io.TFRecordWriter(path_tfrecords_validation) as writer:
    #Lista donde almacenaremos todos los objetos "Example"
    all_example = []
    #Bucle que se encarga de recorrer los pacioentes
    for i,path in enumerate(directorio_entrenamiento):
        #Mostramos por pantalla el progreso que llevamos 
        print_progress(count = i, total = len(directorio_entrenamiento)-1)
        print("\n")
        #Mostramos el Indice y el paciente actual
        print(i,path)
        #Como índica su nombre es el directorio del paciente actual
        directorio_paciente = glob.glob(path + '\\*')
        #Bucle que se encarga de recorrer los ficheros del paciente actual
        for(j,path_paciente) in enumerate(directorio_paciente):
            #print(j,path_paciente)
            #Leemos el fichero relacionado con el tipo "Flair"
            if path_paciente.endswith("flair.nii.gz"):
                #print("Entra en flair")
                flair = glob.glob(path_paciente)
                norm_flair = norm_image_by_patient(flair[0])
            #Leemos el fichero relacionado con el tipo "T2"      
            if path_paciente.endswith("t2.nii.gz"):
                #print("Entra en t2")
                t2 = glob.glob(path_paciente)
                norm_t2 = norm_image_by_patient(t2[0])
            #Leemos el fichero relacionado con el tipo "T1"    
            if path_paciente.endswith("t1.nii.gz"):
                #print("Entra en t1")
                t1 = glob.glob(path_paciente)
                norm_t1 = norm_image_by_patient(t1[0])
            #Leemos el fichero relacionado con el tipo "SEG"
            #Este es especial porque es la etiqueta donde están la segmentación hecha por el médico
            if path_paciente.endswith("seg.nii.gz"):
                #print("Entra en seg")
                label = glob.glob(path_paciente)[0]
                label = sitk.GetArrayFromImage(sitk.ReadImage(label)).astype(np.float32)
            #Leemos el fichero relacionado con el tipo "T1CE"
            if path_paciente.endswith("t1ce.nii.gz"):
                #print("Entra en t1ce")
                t1ce = glob.glob(path_paciente)
                norm_t1ce = norm_image_by_patient(t1ce[0])
        #Comprobamos que cada canal se haya normalizado correctamente        
        if((len(norm_flair) == 0) or (len(norm_t2) == 0) or (len(norm_t1) == 0) or (len(norm_t1ce) == 0)):
            print("ERROR FALTA ALGUNA DE LAS COMPONENTES")
            print(len(norm_flair),len(norm_t2),len(norm_t1),len(norm_t1ce))
        else:
            seq = [norm_flair,norm_t2,norm_t1,norm_t1ce]
            ind = 0
            for depth in range(155):
                ind += 1
                #Procesamos la imagen y la etiqueta
                image_data, label_data, shape = _processing_image_single(seq, label, depth)
                #Convertimos las imagenes y las etiquetas procesadas en el objetos de tipo "Example"
                example = _convert_to_example(image_data, label_data)
                #Serializamos la información y la escribimos en un fichero "TFRecord"
                writer.write(example.SerializeToString())
    print ('Transformación Realizada Completamente!!!')